# Notes: You will have to:
  >1- Make a Copy of this notebook to edit it for your solutions;
  >
  >2- Upload on Gradescope: The `Colab Notebook edited with your solutions`, and a `pdf Report (Discussion questions)`.

# Question 1: MLP Implementation with Numpy

In this exercise, we will explore the construction of a multi-layer perceptron (MLP) by coding it from the ground up. We will develop a deep neural network step-by-step, beginning with a single neuron, progressing to a layer, and ultimately building the entire network. We will then train this network and evaulate on an interesting dataset and try to understand the workings in more details.

## Installing dependencies

In [7]:
# !pip install numpy scikit-learn matplotlib

In [8]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_circles

## Q 1.1 (a): Activation Functions

In this question you will implement the softmax and ReLU activation functions and their derivatives. Both these functions help introduce non-linearity which gives neural network their capacity. We will implement these functions in the `ActivationFunction` class as static methods. You can access the method later using the following code:

```
# For softmax activation
ActivationFunction.softmax(x)

# For calucating the derivative of softmax function
ActivationFunction.softmax_derivative(x)

# For ReLU activation
ActivationFunction.relu(x)

# For calculating the derivative of ReLU function
ActivationFunction.relu_derivative(x)
```

In [9]:
class ActivationFunction:
    @staticmethod
    def softmax(x):
        """
        Compute the Softmax activation function.

        Args:
            x (np.array): Input values

        Returns:
            np.array: Softmax activation values
        """
        ### Write your code here ###
        expo = np.exp(x)
        return expo / np.sum(expo)
        

    @staticmethod
    def softmax_derivative(x):
        """
        Compute the derivative of the Softmax function.

        Args:
            x (np.array): Input values

        Returns:
            np.array: Derivative of Softmax function
        """
        ### Write your code here ###

        exps = np.exp(x - np.max(x))  
        s = exps / np.sum(exps)

        return  np.diag(s) - np.outer(s, s)

    @staticmethod
    def relu(x):
        """
        Compute the ReLU activation function.

        Args:
            x (np.array): Input values

        Returns:
            np.array: ReLU activation values
        """
        ### Write your code here ###
        return np.maximum(0, x)

    @staticmethod
    def relu_derivative(x):
        """
        Compute the derivative of the ReLU activation function.

        Args:
            x (np.array): Input values

        Returns:
            np.array: Derivative of ReLU function
        """
        ### Write your code here ###
        return np.where(x > 0, 1, 0)

## Q 1.1 (b): Neuron Implementation

Neuron is the fundamental building block of MLP. A neuron simulates a biological neuron which can be activated or fired when responding to a stimulus. The equation of a neuron is as follow:

$$ f(x) = a(\mathbf{w} \cdot \mathbf{x} + b) $$

where inputs $\mathbf{x}$ and weights $\mathbf{w}$ are multidimensional vectors, bias $b$ is a scalar, $a(.)$ is an activation function.

**Note:** For the whole exercise you will be receiving inputs of batch size of 1.

In [10]:
class Neuron:
    def __init__(self, input_size, activation_function):
        """
        Initialize a neuron with random weights and bias.

        Args:
            input_size (int): Number of input features
            activation_function (function): Activation function to use
        """
        ### Write your code here ###
        self.activation_function = activation_function
        self.weights = np.random.randn(input_size)
        self.bias = np.random.randn()

    def activate(self, x):
        """
        Compute the activation of the neuron.

        Args:
            x (np.array): Input features

        Returns:
            float: Activation value
        """
        ### Write your code here ###
        return self.activation_function(self.weights @ x + self.bias)

## Q 1.1(c): Layer Implementation

Now we will compose a layer which consists of multiple neurons. Complete initializing the layer and compute the forward pass by sending the input through each neuron of the layer.

In [11]:
class Layer:
    def __init__(self, input_size, output_size, activation_function):
        """
        Initialize a layer of neurons.

        Args:
            input_size (int): Number of input features
            output_size (int): Number of neurons in the layer
            activation_function (function): Activation function to use
        """
        ## Write your code here ##
        self.neurons = [Neuron(input_size, activation_function) for _ in range(output_size)]
    

    def forward(self, x):
        """
        Compute the forward pass through the layer.

        Args:
            x (np.array): Input features

        Returns:
            np.array: Output of the layer
        """
        ### Write your code here ###
        return np.array([neuron.activate(x) for neuron in self.neurons])

## Q 1.1(d) + 2 (a) to (d): Neural Network Construction

Now we have built all the building blocks required to build our neural network. Complete the implementations using the hints given in the code. There are additional functions that will be useful later. Do not modify these.

In [ ]:
import numpy as np

class NeuralNetwork:
    def __init__(self, layer_sizes, hidden_dim, activation_function_hidden, activation_output):
        """
        Initialize a neural network with specified layer sizes.

        Args:
            layer_sizes (list): List of integers representing the size of each layer
            hiden_dim (int): Dimension of hidden layers
            activation_function_hidden (function): Activation function to use for hidden layers
            activation_output (function): Activation function to use for output layer
        """
        self.layers = []
        self.hidden_dim = hidden_dim
        self.activation_function_hidden = activation_function_hidden
        self.activation_output = activation_output
        ## Write your code here ##
        #TODO Define the network architecture in the constructor with exactly 2 layers: one hidden layer (64 neurons)
        # and one output layer (10 units). (1 point)
        # input_size = layer_sizes[0]
        # self.layers.append(Layer(input_size, 64, activation_function_hidden))
        # self.layers.append(Layer(64, 10, activation_output))
        for i in range(len(layer_sizes) - 1):
            input_size = layer_sizes[i]
            output_size = layer_sizes[i + 1]
            
            # Use hidden activation for all layers except the last one
            if i < len(layer_sizes) - 2:
                activation = activation_function_hidden
            else:
                activation = activation_output
            
            self.layers.append(Layer(input_size, output_size, activation))

    def forward(self, x):
        """
        Compute the forward pass through the entire network.
        Args:
            x (np.array): Input features
        Returns:
            np.array: Output of the network
        """
        ## Write your code here ##
        # Implement forward(X) using only NumPy. Apply softmax to the final logits
        
        self.activations = [x] 
        self.pre_activations = []
        
        output = x
        for layer in self.layers:

            weights = np.array([neuron.weights for neuron in layer.neurons])
            biases = np.array([neuron.bias for neuron in layer.neurons])
            z = weights @ output + biases
            self.pre_activations.append(z)
            
            output = layer.forward(output)
            self.activations.append(output)
        

        exp_logits = np.exp(output - np.max(output))
        softmax_output = exp_logits / np.sum(exp_logits)
        
        self.activations[-1] = softmax_output
        
        return softmax_output
        
    def layer_backward(self, delta, layer_index, activations, zs):
        """
        Perform backpropagation for a single layer.

        Args:
            delta (np.array): Gradient of the loss with respect to the output of the current layer
            layer_index (int): Index of the current layer
            activations (list): List of activations from the forward pass
            zs (list): List of z values from the forward pass

        Returns:
            tuple: (delta for the previous layer, gradients for weights, gradients for biases)
        """
        layer = self.layers[layer_index]
        z = zs[layer_index]
        a_prev = activations[layer_index]
        a = activations[layer_index + 1]

        # Compute activation derivative
        if layer_index == len(self.layers) - 1:
            delta = self.softmax_derivative(a) * delta
        else:
            if self.activation_function_hidden == ActivationFunction.relu:
                delta = self.relu_derivative(z) * delta
            else:
                raise ValueError("Unsupported activation function")

        gradient_w = np.outer(delta, a_prev)
        gradient_b = delta

        weights = np.array([neuron.weights for neuron in layer.neurons])

        # Compute delta to pass back to previous layer
        delta_prev = np.dot(weights.T, delta)

        return delta_prev, gradient_w, gradient_b


    def backward(self, x, y, learning_rate):
        """
        Perform backpropagation and update weights.

        Args:
            x (np.array): Input features
            y (np.array): True labels
            learning_rate (float): Learning rate for weight updates
        """
        # Forward pass
        activations = [x]
        zs = []
        for layer in self.layers:
            z = np.array([np.dot(neuron.weights, activations[-1]) + neuron.bias for neuron in layer.neurons])
            zs.append(z)
            activation = layer.forward(activations[-1])
            activations.append(activation)

        # Backward pass
        delta = activations[-1] - y # difference between prediction and true label 
        for i in reversed(range(len(self.layers))): # revert because we start from the end and go back to beginning
            ## Write your code here ##
            delta, gradient_w, gradient_b = self.layer_backward(delta, i, activations, zs)

            # Update weights and biases
            for j, neuron in enumerate(self.layers[i].neurons):
                ## Write your code here ##
                neuron.weights -= learning_rate * gradient_w[j]
                neuron.bias -= learning_rate * gradient_b[j]

    def train(self, X, y, epochs, learning_rate):
        """
        Train the neural network.

        Args:
            X (np.array): Training features
            y (np.array): Training labels
            epochs (int): Number of training epochs
            learning_rate (float): Learning rate for weight updates
        """
        # write your code here
        # train_losses = []
        # for epoch in range(epochs):
        #     epoch_loss = 0
        #     Write your codes here
        #
        #     if epoch % 50 == 0:
        #         print(f"Epoch {epoch}, Loss: {avg_loss:.4f}")
        pass

    def compute_loss(self, predictions, targets):
        "Compute loss for model predictions and ground truth"
        # write your code here
        return loss

    def predict(self, X):
        """
        Make predictions for a set of input features.

        Args:
            X (np.array): Input features

        Returns:
            np.array: Predictions
        """
        return np.array([self.forward(x) for x in X])

# Q 1.2(d): Training and Visualizing Neural Network

In the rest of the exercise we will use the components we built earlier to train a neural network and visualise the loss curve. To make things simple we will be building neural networks with only one hidden layer.

We will use the MNIST dataset for all our experiements which consists of 10 classes.


In [ ]:
# Uncomment the below code to create the dataset. And comment it when submitting to gradescope

# np.random.seed(42) # Do not change
# For easy data loading and processing, we're using torch here. You're not allowed to use torch for Neural Net components though.
def get_train_test_dataset():
    import torch
    from torchvision import datasets, transforms

    # Define a transform to convert images to tensors and normalize
    transform = transforms.Compose([
        transforms.ToTensor()
    ])

    # Load the MNIST dataset
    mnist_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)

    # Take a subset of 1000 samples
    X = mnist_dataset.data[:1000].float() / 255.0  # Normalize pixel values to [0,1]
    y = mnist_dataset.targets[:1000]

    # Creat test / test split

    # to be completed
    # print("Training set shape:", )
    # print("Test set shape :", )

    return X_train, y_train, X_test, y_test



##Let's visualize a few samples

In [ ]:
# Plot a few images

def visualize_mnist_samples(X, y, num_samples=9):
    """Visualize MNIST sample images"""
    plt.figure(figsize=(10, 8))
    for i in range(min(num_samples, len(X))):
        plt.subplot(3, 3, i + 1)
        # Reshape from flattened to 28x28 for visualization
        img = X[i].reshape(28, 28)
        plt.imshow(img, cmap='gray')
        plt.title(f"Label: {y[i]}")
        plt.axis('off')

    plt.suptitle('MNIST Sample Images')
    plt.tight_layout()
    plt.show()

Below you have been given the code to train your neural network. and visualize the decision boundry. Modify the hyperparameters to train and visualize the neural network and report your findings in the practical report. We will be testing our model on the training set. The `plot_decision_boundary` method takes the data created earlier and the neural network you trained to show the decision boundry.

In [ ]:
def train_network(hidden_layer_size, hidden_dim, epochs=10, activation_hidden=None, activation_output=None, learning_rate=0.001):
    input_size, output_size = 28 * 28, 10
    layers = [input_size, hidden_layer_size, output_size]

    nn = NeuralNetwork(layers, hidden_dim, activation_hidden, activation_output)


    # --- Prepare training data ---
    X_train_flat =
    y_train_np =

    # One-hot encode labels
    y_train_onehot =

    # Train
    train_loss = nn.train(X_train_flat, y_train_onehot, epochs=epochs, learning_rate=learning_rate)

    # --- Evaluate on training data ---
    # preds_train =
    # train_accuracy =

    # --- Evaluate on test data ---
    # X_test_flat =
    # y_test_np =
    # preds_test =
    # test_accuracy =

    return nn, train_loss, train_accuracy, test_accuracy

In [ ]:
# Write the code to train and visualize
# Consider commenting this out when submitting to gradescope

# hidden_layer_size =
# hidden_dim =
# epochs =
# activation_hidden =
# activation_output =
# learning_rate =
# nn, train_loss, train_accuracy, test_accuracy = train_network(hidden_layer_size, hidden_dim  epochs, activation_hidden, activation_output, learning_rate)


# Q 1.3: Reporting and analysis
Analyze your trained model and present findings clearly as asked in the instruction PDF.

In [ ]:
# print("Train accuracy: ", train_accuracy)
# print("Test accuracy: ", test_accuracy)

# Training loss curve

In [ ]:
# Plot the training loss across 20 epochs

def plot_loss_curve(train_loss):
  # Write your code here
  pass

## Layer activation visualization

In [ ]:
# ===== Latent Space Visualization =====
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

def get_penultimate_activations(nn, X_batch):
    """Get activations from the last hidden layer"""
    # Process each sample individually
    activations_list = []
    for x in X_batch:
        activations = x
        for layer in nn.layers[:-1]:  # exclude final output layer
            activations = layer.forward(activations)
        activations_list.append(activations)
    return np.array(activations_list)

def visualize_activations(nn, X_test, y_test, digits=(3, 8), method='PCA'):
    """Visualize layer activations using PCA or t-SNE"""
    # Get activations from penultimate layer
    Z = get_penultimate_activations(nn, X_test)

    # Apply dimensionality reduction
    if method == 'PCA':
        reducer = PCA(n_components=2)
    else:  # t-SNE
        reducer = TSNE(n_components=2, random_state=42)

    Z_2d = reducer.fit_transform(Z)

    # Plot visualization
    plt.figure(figsize=(12, 5))

    # Plot 2: Focus on specific digits
    plt.subplot(1, 2, 2)
    mask1 = y_test == digits[0]
    mask2 = y_test == digits[1]

    # write your code here

    plt.title(f'Latent Space: Digits {digits[0]} vs {digits[1]}')

    plt.xlabel('Component 1')
    plt.ylabel('Component 2')
    plt.tight_layout()
    plt.show()

    return Z_2d

## Hyperparemeter discussion
You can try out changing hypermaters (leraning rate, hidden dim, num. of hidden layers) and report how that impact training loss, train/test accuracy

## Question 2: Convolutional Neural Network (using PyTorch)

### Setup

In [ ]:

import math, random, time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# Hyperparameters
learning_rate = 1e-3  # or 3e-4
num_epochs = 50
batch_size = 64
num_classes = 10
validation_split = 0.2



## Part 1: Implementing the CNN

You will build the core components of the CNN, with **both** forward and backward passes.

### (a) Fully Connected Layer
Implement a linear layer:
y = xW^T + b

In [ ]:

class CustomLinear(nn.Module):
    def __init__(self, in_features, out_features, bias=True):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = nn.Parameter(torch.Tensor(out_features, in_features))
        self.bias   = nn.Parameter(torch.Tensor(out_features)) if bias else None
        self.reset_parameters()
        self._cache_x = None

    def reset_parameters(self):
        # Kaiming Uniform (fixed by assignment)
        nn.init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = nn.init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / math.sqrt(fan_in)
            nn.init.uniform_(self.bias, -bound, bound)

    def forward(self, x):
        """
        TODO: Compute y = x W^T + b.
        Hints:
          • Use a batched matmul pattern that preserves leading batch dims.
          • Add bias if present.
        """
        self._cache_x = x
        "WRITE YOUR CODE HERE"
        pass

    def backward(self, grad_output):
        """
        TODO: Compute gradients wrt input, weight, bias.
        param grad_output: gradient wrt output
        Return: (grad_input, grad_weight, grad_bias)
        """
        x = self._cache_x
        "WRITE YOUR CODE HERE"
        pass


### (b) Convolution Layer
Implement a 2D convolution layer with basic operations

In [ ]:

class CustomConv2D(nn.Module):
    """
    2D Convolution (k x k) with manual forward/backward using unfold/fold.

    Shapes and symbols:
      x: input of shape (N, C_in, H, W)
        N = batch size, C_in = input channels, H/W = spatial dims: Heigth/Width
      w: weights of shape (C_out, C_in, k, k), with C_out = output channels
      b: bias of shape (C_out,)
      k: kernel_size
      s: stride
      p: padding

      H_out = floor((H + 2*p - k)/s) + 1
      W_out = floor((W + 2*p - k)/s) + 1
      L = H_out * W_out   # number of sliding-window locations per feature map
    """
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.weight = nn.Parameter(torch.Tensor(out_channels, in_channels, kernel_size, kernel_size))
        self.bias   = nn.Parameter(torch.Tensor(out_channels))
        self.reset_parameters()
        self._cache_x = None

    def reset_parameters(self):
        # Kaiming Uniform (fixed)
        nn.init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        fan_in, _ = nn.init._calculate_fan_in_and_fan_out(self.weight)
        bound = 1 / math.sqrt(fan_in)
        nn.init.uniform_(self.bias, -bound, bound)

    def forward(self, x):
        """
        TODO: Compute conv output using F.unfold and a batched dot-product.
        Hints:
          • Unfold to shape (N, C_in*k*k, L) where L is number of locations.
          • Reshape weights to (C_out, C_in*k*k).
          • Add bias per output channel.
          • Reshape to (N, C_out, H_out, W_out).
        """
        self._cache_x = x
        "WRITE YOUR CODE HERE"
        pass

    def backward(self, grad_output):
        """
        TODO: Compute gradients wrt input, weight, bias.
        Hints:
          • Reuse unfolded patches.
          • grad_weight: correlate grad_output with unfolded input patches.
          • grad_bias: sum grad_output over batch and spatial locations.
          • grad_input: map grads back via an unfolded representation and fold.
        Return: (grad_input, grad_weight, grad_bias)
        """
        x = self._cache_x
        "WRITE YOUR CODE HERE"
        pass


### (c) Max Pooling Layer
Implement a max pooling layer

In [ ]:

class CustomMaxPool2D(nn.Module):
    """
    Max Pooling (k x k) with manual forward/backward using unfold/fold.

    Shapes and symbols:
      x: input of shape (N, C, H, W)
        N = batch size, C = channels, H/W = spatial dims: Height, Width
      k: kernel_size (pool window is k x k)
      s: stride
      p: padding

      H_out = floor((H + 2*p - k)/s) + 1
      W_out = floor((W + 2*p - k)/s) + 1
      L = H_out * W_out   # number of sliding-window locations per feature map
    """
    def __init__(self, kernel_size, stride=None, padding=0):
        super().__init__()
        self.kernel_size = kernel_size
        self.stride = stride if stride is not None else kernel_size
        self.padding = padding
        self._cache_shape = None
        self._cache_indices = None

    def forward(self, x):
        """
        TODO: Max over k*k windows.
        Hints:
          • Unfold x → patches of shape (N, C*k*k, L)
          • View → (N, C, k*k, L)
          • Take max over the k*k dimension → outputs (N, C, L) and argmax indices (N, C, L)
          • Cache argmax indices + shapes.
          • Reshape output reshaped to (N, C, H_out, W_out).
        """
        "WRITE YOUR CODE HERE"
        pass

    def backward(self, grad_output):
        """
        TODO: Implement max pooling backward (route grads to maxima only).
        Inputs: grad_output: (N, C, H_out, W_out)
        Hints:
          • Make zeros tensor grad_unfold of shape (N, C, k*k, L)
          • Scatter grad_output into grad_unfold at the argmax positions from forward
          • View grad_unfold → (N, C*k*k, L)
          • Fold back to image space → grad_input of shape (N, C, H, W)

        Return: grad_input(N, C, H, W)
        """
        "WRITE YOUR CODE HERE"
        pass


### (d) Cross-Entropy Loss [2 points]
Implement cross-entropy loss with numerical stability

In [ ]:

class CustomCrossEntropyLoss(nn.Module):
    def __init__(self, reduction="mean"):
        super().__init__()
        self.reduction = reduction
        self._cache = None

    def forward(self, logits, targets):
        """
        TODO: Compute numerically-stable cross-entropy.
        Hints:
          • Subtract max per row before exponentiation.
          • Convert to probabilities; pick class probs at targets.
          • Apply -log(...) and reduction (mean by default).
          • Cache what's needed for backward.
        """
        "WRITE YOUR CODE HERE"
        pass

    def backward(self, logits=None, targets=None):
        """
        TODO: Compute gradient w.r.t. logits.
        Hints:
          • If `logits` and `targets` are provided, recompute softmax probs stably.
            Otherwise, use cached probs/targets from forward().
          • Divide by batch size if reduction == 'mean'.
        Return: grad_logits
        """
        "WRITE YOUR CODE HERE"
        pass




## Part 2: Training the CNN

Build and train the CNN for MNIST using **your custom layers**.

### (a) Training pipeline code
Address `TODO` in:
1. **Backward pass section** — explicit chaining with your `backward()` methods.  
2. **Parameter updates section** — manual updates using SGD-style with `torch.no_grad()`.

### (b) Training and experiments
Train your CNN on MNIST and run the following configurations:
1. **Activations (2):** ReLU, Tanh.  
2. **Learning rates (2):** 1e-3, 3e-4.

Use consistent hyperparameters across runs. This yields 4 configurations.


In [ ]:
def get_activation(name):
    if name.lower() == "relu":
        return F.relu, lambda z: (z > 0).to(z.dtype)
    elif name.lower() == "tanh":
        return torch.tanh, lambda z: 1 - torch.tanh(z)**2
    else:
        raise ValueError("Unknown activation: " + name)

class CNN(nn.Module):
    def __init__(self, activation_name="relu", num_classes=10):
        super().__init__()
        self.ACT, self.ACT_PRIME = get_activation(activation_name)

        self.conv1 = CustomConv2D(1, 32, 5)
        self.conv2 = CustomConv2D(32, 64, 5)
        self.max1  = CustomMaxPool2D(2, stride=2)
        self.max2  = CustomMaxPool2D(2, stride=2)
        self.fc1   = CustomLinear(64*4*4, 512)
        self.fc2   = CustomLinear(512, num_classes)

        self.criterion = CustomCrossEntropyLoss()

    def forward(self, x):
        z1 = self.conv1(x)
        a1 = self.ACT(z1)
        p1 = self.max1(a1)

        z2 = self.conv2(p1)
        a2 = self.ACT(z2)
        p2 = self.max2(a2)

        flat = torch.flatten(p2, start_dim=1)
        z3 = self.fc1(flat)
        a3 = self.ACT(z3)
        logits = self.fc2(a3)

        # caches needed for manual backward
        caches = (z1, a1, p1, z2, a2, p2, z3, a3)
        return logits, caches

### Data (MNIST)

In [ ]:

full_train = datasets.MNIST(root="./data_CNN", train=True, download=True, transform=ToTensor())
test_ds    = datasets.MNIST(root="./data_CNN", train=False, download=True, transform=ToTensor())

train_size = int((1 - validation_split) * len(full_train))
val_size   = len(full_train) - train_size
train_ds, val_ds = random_split(full_train, [train_size, val_size])

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=batch_size, shuffle=False)
test_loader  = DataLoader(test_ds, batch_size=batch_size, shuffle=False)

print(f"Train/Val/Test sizes: {len(train_ds)}/{len(val_ds)}/{len(test_ds)}")



### Utilities

In [ ]:

@torch.no_grad()
def accuracy_from_logits(logits, labels):
    return (logits.argmax(dim=1) == labels).float().mean().item()

def sgd_update_(param, grad, lr):
    param -= lr * grad
    return param



### Training Pipeline code
Address the TODOs in the code below
> - Complete the BACKWARD PASS (explicit chaining with your `.backward` methods).
> - Uncomment the PARAMETER UPDATES (manual SGD with no_grad)


In [ ]:

def train_one_config(activation_name="relu", epochs=num_epochs, lr=learning_rate, num_classes=num_classes):
    model = CNN(activation_name=activation_name, num_classes=num_classes).to(device)
    criterion = model.criterion

    train_losses, val_losses = [], []
    train_accs,   val_accs   = [], []

    for epoch in range(epochs):
        model.train()
        total_train_loss, total_train_correct, total_train_examples = 0.0, 0, 0

        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)

            # ---------- Forward ----------
            logits, caches = model(xb)
            loss = criterion(logits, yb)

            # Unpack caches (pre/post activations)
            z1, a1, p1, d1, z2, a2, p2, d2, z3, a3 = caches

            # ==========================================================================
            # BACKWARD PASS (explicit chaining with your .backward methods)
            # TODO: Implement the manual chain below and produce gradients with names:
            #       grad_fc2_w, grad_fc2_b, grad_fc1_w, grad_fc1_b,
            #       grad_conv2_w, grad_conv2_b, grad_conv1_w, grad_conv1_b
            #
            # Hints:
            #   1- Check PARAMETER UPDATES codes below for ideas and variable names
            #   2- Chain in the following way:
            #       - dL/dlogits from Cross-Entropy Loss
            #       - FC2 backward
            #       - Activation after FC1
            #       - FC1 backward
            #       - Reshape to feature map
            #       - MaxPool2 backward
            #       - Activation after Conv2
            #       - Conv2 backward
            #       - MaxPool1 backward
            #       - Activation after Conv1
            #       - Conv1 backward
            "WRITE YOUR CODE HERE"


            # ===========================================
            # PARAMETER UPDATES (manual SGD with no_grad)
            # TODO: UNCOMMENT THE CODES BELOW, DO NOT CHANGE
            # ===========================================
            # with torch.no_grad():
            #     # FC2
            #     model.fc2.weight.copy_(sgd_update_(model.fc2.weight, grad_fc2_w, lr))
            #     if model.fc2.bias is not None:
            #         model.fc2.bias.copy_(sgd_update_(model.fc2.bias, grad_fc2_b, lr))
            #     # FC1
            #     model.fc1.weight.copy_(sgd_update_(model.fc1.weight, grad_fc1_w, lr))
            #     if model.fc1.bias is not None:
            #         model.fc1.bias.copy_(sgd_update_(model.fc1.bias, grad_fc1_b, lr))
            #     # Conv2
            #     model.conv2.weight.copy_(sgd_update_(model.conv2.weight, grad_conv2_w, lr))
            #     model.conv2.bias.copy_(sgd_update_(model.conv2.bias, grad_conv2_b, lr))
            #     # Conv1
            #     model.conv1.weight.copy_(sgd_update_(model.conv1.weight, grad_conv1_w, lr))
            #     model.conv1.bias.copy_(sgd_update_(model.conv1.bias, grad_conv1_b, lr))


            # ---------- Train metrics ----------
            with torch.no_grad():
                total_train_loss += loss.item()
                total_train_examples += yb.size(0)
                total_train_correct += (logits.argmax(1) == yb).sum().item()

        train_losses.append(total_train_loss / len(train_loader))
        train_accs.append(total_train_correct / total_train_examples)

        # ---------- Validation ----------
        model.eval()
        val_loss_sum, val_correct, val_examples = 0.0, 0, 0
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                logits, _ = model(xb)
                loss = criterion(logits, yb)
                val_loss_sum += loss.item()
                val_examples += yb.size(0)
                val_correct += (logits.argmax(1) == yb).sum().item()

        val_losses.append(val_loss_sum / len(val_loader))
        val_accs.append(val_correct / val_examples)

        print(f"Epoch {epoch+1:02d} | "
              f"Train Loss {train_losses[-1]:.4f}  Acc {train_accs[-1]*100:.2f}% | "
              f"Val Loss {val_losses[-1]:.4f}  Acc {val_accs[-1]*100:.2f}%")

    # ---------- Test ----------
    model.eval()
    test_correct, test_total = 0, 0
    with torch.no_grad():
        for xb, yb in test_loader:
            xb, yb = xb.to(device), yb.to(device)
            logits, _ = model(xb)
            test_correct += (logits.argmax(1) == yb).sum().item()
            test_total += yb.size(0)
    test_acc = 100.0 * test_correct / test_total
    print(f"Test Accuracy: {test_acc:.2f}%")

    return {
        "train_loss": train_losses, "val_loss": val_losses,
        "train_acc": train_accs,   "val_acc": val_accs,
        "test_acc": test_acc
    }


### Training, and experiments
Train your CNN on MNIST and run the following configurations:
1. **Activations (2):** ReLU, Tanh.  
2. **Learning rates (2):** 1e-3, 3e-4.

Use consistent hyperparameters across runs. This yields 4 configurations:
- ReLU / LR = 1e-3
- ReLU / LR = 3e-4
- Tanh / LR = 1e-3
- Tanh / LR = 3e-4

In [ ]:
# Training, and experiments : 4 configurations in total
"WRITE YOUR CODE HERE"


## Part 3: Results and Discussions

1. **Plots:** Submit clear plots of training and validation **accuracy** and **loss** for each configuration. In addition, include the **test-set accuracy** on these plots. **Discuss results for each configuration**.


In [ ]:
# Plots
"WRITE YOUR CODE HERE"

  
2. **Write a short discussion:** (~1 page) reflecting on:
   - Which **activations/learning rates** choices performed best and why?
   - Any signs of over/underfitting you observed.

Note: Please upload the discussion write-ups of all sections of **Part 3** as a single PDF file on Gradescope.